In [1]:
import numpy as np
import pandas as pd
import scipy

In [25]:
from statsmodels.stats import weightstats

In [47]:
def write_ans(ans,name,path):
    with open(path + name +'.txt','w') as func:
        func.write(str(ans))

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data.isnull().sum().sum() # Нет пропущенных данных

0L

In [10]:
X_data = data.drop(['Patient_id','Diagnosis'],axis=1)
y_data = data.Diagnosis

In [15]:
print np.array(y_data.value_counts().keys())

['early neoplasia' 'normal' 'cancer']


In [37]:
#for i in range(X_data.shape[1]):
#cm = weightstats.CompareMeans(weightstats.DescrStatsW(X_data.ix[y_data == 'normal',0]),
 #                             weightstats.DescrStatsW(X_data.ix[y_data == 'normal',0]))

(0.0, 1.0, 46.0)

### Часть 1: применение t-критерия Стьюдента

In [68]:
normal_early_p = []
for i in range(X_data.shape[1]):
    normal_early_p.append(scipy.stats.ttest_ind(X_data.ix[y_data == 'normal',i],
                                                X_data.ix[y_data == 'early neoplasia',i],
                                               equal_var = False).pvalue)

In [69]:
ans1 = (np.array(normal_early_p) < 0.05).sum() # 1 ответ OK
ans1

1575

In [71]:
early_cancer_p = []
for i in range(X_data.shape[1]):
    early_cancer_p.append(scipy.stats.ttest_ind(X_data.ix[y_data == 'early neoplasia',i],
                                                X_data.ix[y_data == 'cancer',i],
                                               equal_var = False).pvalue)

In [72]:
ans2 = (np.array(early_cancer_p) < 0.05).sum() # 2 ответ OK
ans2

3490

In [61]:
path1 = 'D:\\Leon Projects\\Python\\Postroenie Vivodov po dannim 4 kurs\\Week4\\Task 1 bioinformatic task\\ans\\'

In [73]:
write_ans(ans1,'bio.1_1',path1)
write_ans(ans2,'bio.1_2',path1)

### Часть 2: поправка методом Холма

In [74]:
import statsmodels.stats.multitest as smm

In [75]:
normal_early_p = np.array(normal_early_p)
early_cancer_p = np.array(early_cancer_p)

In [166]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_early_p, alpha=0.025,method='holm')

In [169]:
p_corrected[reject == True] / normal_early_p[reject == True]

array([ 15747.,  15748.])

In [170]:
reject, p_corrected, a1, a2 = smm.multipletests(early_cancer_p, alpha=0.025,method='holm')

In [185]:
z = zip(X_data.ix[y_data == 'early neoplasia',reject == True].mean(),
   X_data.ix[y_data == 'cancer',reject == True].mean())

In [188]:
fold_change = [x[1]/x[0] if x[0]<x[1] else -x[0]/x[1] for x in z]

In [ ]:
def fold_change()

In [190]:
(abs(np.array(fold_change)) > 1.5).sum()

77

In [165]:
write_ans(2,'bio.2_1',path1)

In [191]:
write_ans(77,'bio.2_2',path1)

### Часть 3: поправка методом Бенджамини-Хохберга

In [193]:
reject, p_corrected, a1, a2 = smm.multipletests(normal_early_p, alpha=0.025,method='fdr_bh')
z = zip(X_data.ix[y_data == 'normal',reject == True].mean(),
   X_data.ix[y_data == 'early neoplasia',reject == True].mean())
fold_change = [x[1]/x[0] if x[0]<x[1] else -x[0]/x[1] for x in z]
ans5 = (abs(np.array(fold_change)) > 1.5).sum()
ans5

4

In [194]:
write_ans(ans5,'bio.3_1',path1)

In [197]:
reject, p_corrected, a1, a2 = smm.multipletests(early_cancer_p, alpha=0.025,method='fdr_bh')
z = zip(X_data.ix[y_data == 'early neoplasia',reject == True].mean(),
   X_data.ix[y_data == 'cancer',reject == True].mean())
fold_change = [x[1]/x[0] if x[0]<x[1] else -x[0]/x[1] for x in z]
ans6 = (abs(np.array(fold_change)) > 1.5).sum()
ans6

524

In [199]:
write_ans(ans6,'bio.3_2',path1)